In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import pyplot
%matplotlib inline

In [35]:
#Daten einlesen
df_orig = pd.read_parquet("event_ano_new.parquet.gzip")

In [22]:
df_orig.head()

,event_timestamp,message_number,message_category,message_severity,an_title,an_description,an_line,an_cell,an_robot
0,2016-07-04 16:19:35,10010,Operational,Information,NaN,NaN,line_0,cell_0,robot_0
1,2016-07-04 16:19:35,10017,Operational,Information,NaN,NaN,line_0,cell_0,robot_0
2,2016-07-04 16:19:35,10016,Operational,Information,NaN,NaN,line_0,cell_0,robot_0
3,2016-07-04 16:19:34,10015,Operational,Information,NaN,NaN,line_0,cell_0,robot_0
4,2016-07-04 16:19:34,10012,Operational,Information,NaN,NaN,line_0,cell_0,robot_0


In [36]:
df = df_orig.set_index("event_timestamp")
#del df['Unnamed: 0']  # gab es bei mir nicht
df.sort_index(inplace=True)

In [40]:
line_mapping = {}
for i in range(0,26):
    line_mapping.update({"line_{}".format(i):i})

#df["an_line"] = df.an_line.apply(lambda x: line_mapping[x] if x in line_mapping.keys() else x)
    
# hier eine alternative Lösung - wenn möglich auf "apply" verzichten (Performance)
line_mapping2 = {}
for k, v in [(l, int(l.split("_")[1])) for l in list(df.an_line.unique())]:
    line_mapping2[k] = v

# ich empfehle Ihnen, nicht die Originalspalten zu überschreiben
# so können Sie diese Zelle mehrfach ausführen und müssen nicht wieder nach oben springen
df["an_line_no"] = df.an_line.map(line_mapping2)  
# das gilt für die folgenden Zellen auch.

In [41]:
cell_mapping = {}
for i in range(0,13):
    cell_mapping.update({"cell_{}".format(i):i})

df["an_cell"] = df.an_cell.apply(lambda x: cell_mapping[x] if x in cell_mapping.keys() else x)

In [42]:
robot_mapping = {}
for i in range(0,188):
    robot_mapping.update({"robot_{}".format(i):i})

df["an_robot"] = df.an_robot.apply(lambda x: robot_mapping[x] if x in robot_mapping.keys() else x)

In [43]:
message_severity_mapping = {"Error":1,"Warning":2,"Information":3}

df["message_severity"] = df.message_severity.apply(lambda x: message_severity_mapping[x] if x in message_severity_mapping.keys() else x)

In [46]:
def dfExtractMethod(df,extract_value):
    #df_extract = df.loc[df['an_line'] == extract_value]
    df_extract = df.loc[df['an_line_no'] == extract_value]  # hier angepasst auf line_no
    return df_extract

In [47]:
df_extract = dfExtractMethod(df,1)
df_extract.head()

,message_number,message_category,message_severity,an_title,an_description,an_line,an_cell,an_robot,an_line_no
event_timestamp,,,,,,,,,
2016-02-17 05:47:31,10010,Operational,3,NaN,NaN,line_1,1,2,1
2016-02-17 05:47:34,10011,Operational,3,NaN,NaN,line_1,1,2,1
2016-02-17 05:47:45,10010,Operational,3,NaN,NaN,line_1,1,18,1
2016-02-17 05:47:46,10052,Operational,3,NaN,NaN,line_1,1,2,1
2016-02-17 05:47:47,10053,Operational,3,NaN,NaN,line_1,1,2,1


In [48]:
df_extract_index = df_extract.reset_index()
df_extract_index.head()

,event_timestamp,message_number,message_category,message_severity,an_title,an_description,an_line,an_cell,an_robot,an_line_no
0,2016-02-17 05:47:31,10010,Operational,3,NaN,NaN,line_1,1,2,1
1,2016-02-17 05:47:34,10011,Operational,3,NaN,NaN,line_1,1,2,1
2,2016-02-17 05:47:45,10010,Operational,3,NaN,NaN,line_1,1,18,1
3,2016-02-17 05:47:46,10052,Operational,3,NaN,NaN,line_1,1,2,1
4,2016-02-17 05:47:47,10053,Operational,3,NaN,NaN,line_1,1,2,1


In [74]:
# ich verstehe nicht komplett die Methode unten. 
# Soll folgendes erreicht werden? für eine Nachricht die zehn vorherigen und zehn folgenden Nachrichtentypen bekommen?
# Falls ja, würde ich dies über "shift" lösen

df_pre_post = df_extract.copy()  # nur für debug!
no_of_messages = 5
for i in range(1,no_of_messages + 1):
    df_pre_post['msg_svr_pre_{}'.format(i)] = df_pre_post.shift(i).message_severity
    df_pre_post['msg_svr_post_{}'.format(i)] = df_pre_post.shift(-1*i).message_severity
    
df_pre_post.head()

,message_number,message_category,message_severity,an_title,an_description,an_line,an_cell,an_robot,an_line_no,msg_svr_pre_1,msg_svr_post_1,msg_svr_pre_2,msg_svr_post_2,msg_svr_pre_3,msg_svr_post_3,msg_svr_pre_4,msg_svr_post_4,msg_svr_pre_5,msg_svr_post_5
event_timestamp,,,,,,,,,,,,,,,,,,,
2016-02-17 05:47:31,10010,Operational,3,NaN,NaN,line_1,1,2,1,NaN,3,NaN,3,NaN,3,NaN,3,NaN,3
2016-02-17 05:47:34,10011,Operational,3,NaN,NaN,line_1,1,2,1,3,3,NaN,3,NaN,3,NaN,3,NaN,3
2016-02-17 05:47:45,10010,Operational,3,NaN,NaN,line_1,1,18,1,3,3,3,3,NaN,3,NaN,3,NaN,3
2016-02-17 05:47:46,10052,Operational,3,NaN,NaN,line_1,1,2,1,3,3,3,3,3,3,NaN,3,NaN,3
2016-02-17 05:47:47,10053,Operational,3,NaN,NaN,line_1,1,2,1,3,3,3,3,3,3,3,3,NaN,3


In [75]:
# danach können Sie zählen, wie viele Nachrichten der verschiedenen severity-Klassen vorher / nachher aufgetreten sind

pre_column_names = ['msg_svr_pre_{}'.format(i) for i in range(1, no_of_messages + 1)]
post_column_names = ['msg_svr_post_{}'.format(i) for i in range(1, no_of_messages + 1)]

for i in range(1, 4):
    df_pre_post['pre_no_srv_{}'.format(i)] = (df_pre_post[pre_column_names]==i).sum(axis=1)  # axis muss so gesetzt werden!
    df_pre_post['post_no_srv_{}'.format(i)] = (df_pre_post[post_column_names]==i).sum(axis=1) 

    
# in den letzten 6 Spalten stehen nun die aggregierten Werte,
# d.h. Anzahl Nachrichten je Severity-Klasse im angegebenen Zeitfenster (bei mir 5 Nachrichten vorher und nachher,
# das ist steuerbar über no_of_messages)
df_pre_post.head(50)

,message_number,message_category,message_severity,an_title,an_description,an_line,an_cell,an_robot,an_line_no,msg_svr_pre_1,...,msg_svr_pre_4,msg_svr_post_4,msg_svr_pre_5,msg_svr_post_5,pre_no_srv_1,post_no_srv_1,pre_no_srv_2,post_no_srv_2,pre_no_srv_3,post_no_srv_3
event_timestamp,,,,,,,,,,,,,,,,,,,,,
2016-02-17 05:47:31,10010,Operational,3,NaN,NaN,line_1,1,2,1,NaN,...,NaN,3,NaN,3,0,0,0,0,0,5
2016-02-17 05:47:34,10011,Operational,3,NaN,NaN,line_1,1,2,1,3,...,NaN,3,NaN,3,0,0,0,0,1,5
2016-02-17 05:47:45,10010,Operational,3,NaN,NaN,line_1,1,18,1,3,...,NaN,3,NaN,3,0,0,0,0,2,5
2016-02-17 05:47:46,10052,Operational,3,NaN,NaN,line_1,1,2,1,3,...,NaN,3,NaN,3,0,0,0,0,3,5
2016-02-17 05:47:47,10053,Operational,3,NaN,NaN,line_1,1,2,1,3,...,3,3,NaN,3,0,0,0,0,4,5
2016-02-17 05:47:47,10156,Operational,3,NaN,NaN,line_1,1,2,1,3,...,3,3,3,3,0,0,0,0,5,5
2016-02-17 05:47:48,10011,Operational,3,NaN,NaN,line_1,1,18,1,3,...,3,3,3,3,0,0,0,0,5,5
2016-02-17 05:47:55,10271,Operational,3,NaN,NaN,line_1,1,2,1,3,...,3,3,3,3,0,0,0,0,5,5
2016-02-17 05:48:00,10052,Operational,3,NaN,NaN,line_1,1,18,1,3,...,3,3,3,3,0,0,0,0,5,5


In [11]:
def dfExtractErrorWarningInformationAnd10RowsUpAndDownMethod(df,extract_value):
    number = 0
    size = df_extract_index.size
    k = range(1,11)
    data1 = []

    for i in df:
        data1.append(i)

    for j in df.values:
        if(j[3] == extract_value):
            data2 = []
            data3 = []
            data4 = []
            for k in range(1,11):
                if(number+k <= size and number+k >= 0):
                    data2.append(df.values[number-k])
                    data3.append(j)
                    data4.append(df.values[number+k])
                
                if(k == 10):                    
                    data1.append(data2)
                    data1.append(data3[0])
                    data1.append(data4)
        else:
            number = number + 1
            continue
    print("Fertig")
    return(data1)

In [1]:
ErrorsPerLinesPattern = dfExtractErrorWarningInformationAnd10RowsUpAndDownMethod(df_extract_index,1)
myErrorsPerLinesPatternArray = np.asarray(ErrorsPerLinesPattern)
df_ErrorsPerLinesPattern = pd.DataFrame(myErrorsPerLinesPatternArray)
df_ErrorsPerLinesPattern.to_parquet("D:/Eclipse Python/DataScience/res/event_ano_new1.parquet.gzip", compression='gzip')

NameError: name 'dfExtractErrorWarningInformationAnd10RowsUpAndDownMethod' is not defined